In [ ]:
import pandas as pd
from source import image_id_converter as img_idc
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import einops as eo
import pathlib as pl

import matplotlib.cm as cm
from matplotlib import collections  as mc
from matplotlib import animation
%matplotlib inline

from scipy.stats import norm
from scipy.stats import entropy

import pandas as pd
import pickle
from PIL import Image
from time import time as timer
#import umap

from IPython.display import HTML
from IPython.display import Audio
import IPython

import tqdm.auto as tqdm

import torch
from torchvision import datasets, transforms
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchvision import transforms

import sys
import einops as eo

from collections import defaultdict
import random


import os
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
from pathlib import Path
import tempfile

from collections import defaultdict
import random


### Function to Check Image Size

In [ ]:
def analyze_image_sizes(image_directory, extensions=('.jpg', '.jpeg', '.tif', '.tiff')):
   """
   Analyze image sizes in a directory to help determine appropriate target_size.
   
   Args:
       image_directory (str): Path to directory containing images
       extensions (tuple): Allowed file extensions
       
   Returns:
       dict: Dictionary containing size analysis results
   """
   # Get all image paths
   image_paths = load_image_paths(image_directory, extensions)
   
   if not image_paths:
       print("No images found in directory")
       return None
   
   sizes = []
   widths = []
   heights = []
   failed_images = []
   
   print(f"Analyzing {len(image_paths)} images...")
   
   # Analyze each image
   for i, image_path in enumerate(image_paths):
       try:
           with Image.open(image_path) as img:
               width, height = img.size
               sizes.append((width, height))
               widths.append(width)
               heights.append(height)
       except Exception as e:
           failed_images.append((image_path, str(e)))
           print(f"Failed to read {image_path}: {e}")
       
       # Progress indicator for large datasets
       if (i + 1) % 100 == 0:
           print(f"Processed {i + 1}/{len(image_paths)} images...")
   
   if not sizes:
       print("No valid images found")
       return None
   
   # Calculate statistics
   unique_sizes = list(set(sizes))
   all_same_size = len(unique_sizes) == 1
   
   min_width = min(widths)
   max_width = max(widths)
   avg_width = sum(widths) / len(widths)
   
   min_height = min(heights)
   max_height = max(heights)
   avg_height = sum(heights) / len(heights)
   
   # Calculate quartiles
   import numpy as np
   width_q25, width_median, width_q75 = np.percentile(widths, [25, 50, 75])
   height_q25, height_median, height_q75 = np.percentile(heights, [25, 50, 75])
   
   min_size = (min_width, min_height)
   max_size = (max_width, max_height)
   avg_size = (avg_width, avg_height)
   
   # Create results dictionary
   results = {
       'total_images': len(image_paths),
       'valid_images': len(sizes),
       'failed_images': len(failed_images),
       'all_same_size': all_same_size,
       'unique_sizes_count': len(unique_sizes),
       'min_size': min_size,
       'max_size': max_size,
       'avg_size': avg_size,
       'min_width': min_width,
       'max_width': max_width,
       'avg_width': avg_width,
       'min_height': min_height,
       'max_height': max_height,
       'avg_height': avg_height,
       'width_q25': width_q25,
       'width_median': width_median,
       'width_q75': width_q75,
       'height_q25': height_q25,
       'height_median': height_median,
       'height_q75': height_q75,
       'failed_images': failed_images
   }
   
   # Print summary
   print("\n" + "="*50)
   print("IMAGE SIZE ANALYSIS SUMMARY")
   print("="*50)
   print(f"Total images found: {results['total_images']}")
   print(f"Valid images: {results['valid_images']}")
   print(f"Failed to read: {results['failed_images']}")
   print(f"\nAll images same size: {'Yes' if all_same_size else 'No'}")
   print(f"Number of unique sizes: {results['unique_sizes_count']}")
   
   print(f"\nSize ranges:")
   print(f"  Minimum size: {min_size[0]} x {min_size[1]}")
   print(f"  Maximum size: {max_size[0]} x {max_size[1]}")
   print(f"  Average size: {avg_size[0]:.1f} x {avg_size[1]:.1f}")
   
   print(f"\nWidth range: {min_width} - {max_width} (avg: {avg_width:.1f})")
   print(f"Width quartiles: Q25={width_q25:.1f}, Median={width_median:.1f}, Q75={width_q75:.1f}")
   print(f"Height range: {min_height} - {max_height} (avg: {avg_height:.1f})")
   print(f"Height quartiles: Q25={height_q25:.1f}, Median={height_median:.1f}, Q75={height_q75:.1f}")
   
   if results['failed_images']:
       print(f"\nFailed images:")
       for path, error in results['failed_images'][:5]:  # Show first 5 failures
           print(f"  {path}: {error}")
       if len(results['failed_images']) > 5:
           print(f"  ... and {len(results['failed_images']) - 5} more")
   
   # Suggest target size
   if all_same_size:
       print(f"\nRecommendation: Use target_size={min_size} (all images are the same size)")
   else:
       # Suggest a reasonable target size based on minimum dimensions
       suggested_size = min(min_width, min_height)
       # Round to common sizes
       common_sizes = [28, 32, 64, 128, 224, 256, 512]
       suggested_size = min(common_sizes, key=lambda x: abs(x - suggested_size))
       print(f"\nRecommendation: Consider target_size=({suggested_size}, {suggested_size})")
       print(f"  (Based on minimum dimension and common image sizes)")
   
   return results

### Functions to Get Image File Paths and Split them into Training and Validation Set

In [ ]:
def load_image_paths(image_directory, extensions=('.jpg', '.jpeg', '.tif', '.tiff')):
    """
    Load all image file paths from a directory.
    
    Args:
        image_directory (str): Path to directory containing images
        extensions (tuple): Allowed file extensions
        
    Returns:
        list: Sorted list of image file paths
    """
    image_paths = []
    image_dir = Path(image_directory)
    
    if not image_dir.exists():
        raise FileNotFoundError(f"Directory {image_directory} does not exist")
    
    for ext in extensions:
        # Find files with current extension (case insensitive)
        image_paths.extend(image_dir.glob(f"*{ext}"))
        image_paths.extend(image_dir.glob(f"*{ext.upper()}"))
    
    # Sort paths to ensure consistent ordering
    image_paths = sorted([str(path) for path in image_paths])
    
    print(f"Found {len(image_paths)} images in {image_directory}")
    return image_paths

In [ ]:

def split_train_validation(image_paths, labels, train_ratio=0.8, random_seed=42):
    """
    Split image paths into training and validation sets with balanced class distribution.
    
    Args:
        image_paths (list): List of image file paths
        labels (list): List of class labels corresponding to image paths
        train_ratio (float): Proportion for training set (default 0.8)
        random_seed (int): Random seed for reproducibility
        
    Returns:
        tuple: (train_paths, train_labels, val_paths, val_labels)
    """
    random.seed(random_seed)
    
    # Group paths by class
    class_groups = defaultdict(list)
    for path, label in zip(image_paths, labels):
        class_groups[label].append(path)
    
    train_paths, train_labels = [], []
    val_paths, val_labels = [], []
    
    # Split each class maintaining proportion
    for class_label, paths in class_groups.items():
        # Shuffle paths for this class
        shuffled_paths = paths.copy()
        random.shuffle(shuffled_paths)
        
        # Calculate split point
        n_train = int(len(shuffled_paths) * train_ratio)
        
        # Split paths
        class_train_paths = shuffled_paths[:n_train]
        class_val_paths = shuffled_paths[n_train:]
        
        # Add to final lists
        train_paths.extend(class_train_paths)
        train_labels.extend([class_label] * len(class_train_paths))
        val_paths.extend(class_val_paths)
        val_labels.extend([class_label] * len(class_val_paths))
    
    # Final shuffle to mix classes
    combined_train = list(zip(train_paths, train_labels))
    combined_val = list(zip(val_paths, val_labels))
    random.shuffle(combined_train)
    random.shuffle(combined_val)
    
    train_paths, train_labels = zip(*combined_train) if combined_train else ([], [])
    val_paths, val_labels = zip(*combined_val) if combined_val else ([], [])
    
    print(f"Training set: {len(train_paths)} images")
    print(f"Validation set: {len(val_paths)} images")
    
    return list(train_paths), list(train_labels), list(val_paths), list(val_labels)

### Functions Data Loading Pipeline

In [ ]:
def convert_to_grayscale(image):
    """
    Convert PIL image to grayscale.
    
    Args:
        image (PIL.Image): Input image
        
    Returns:
        PIL.Image: Grayscale image
    """
    return image.convert('L')


def apply_aging_effect(image):
    """
    Apply aging effects to PIL image (adapted from your existing function).
    
    Args:
        image (PIL.Image): Input image
        
    Returns:
        PIL.Image: Aged image
    """
    # Ensure image is in RGB mode for aging effects
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    # Heavy JPEG compression using temporary file
    with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as temp_file:
        temp_path = temp_file.name
        image.save(temp_path, 'JPEG', quality=15)
        image = Image.open(temp_path)
        os.unlink(temp_path)  # Clean up temp file
    
    # Significant brightness reduction
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(0.8)
    
    # Low contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(0.9)
    
    # Add significant noise
    img_array = np.array(image)
    noise = np.random.normal(0, 0.1, img_array.shape).astype(np.uint8)
    img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    image = Image.fromarray(img_array)
    
    # Strong blur
    image = image.filter(ImageFilter.GaussianBlur(radius=1.5))
    
    return image


def load_and_preprocess_image(image_path, target_size=(28, 28), convert_grayscale=True, apply_aging=False):
    """
    Load and preprocess a single image.
    
    Args:
        image_path (str): Path to image file
        target_size (tuple): Target size for resizing (width, height)
        convert_grayscale (bool): Whether to convert to grayscale
        apply_aging (bool): Whether to apply aging effects
        
    Returns:
        PIL.Image: Preprocessed image
    """
    try:
        # Load image
        image = Image.open(image_path)
        
        # Apply aging effects first (works best on RGB images)
        if apply_aging:
            image = apply_aging_effect(image)
        
        # Convert to grayscale if requested
        if convert_grayscale:
            image = convert_to_grayscale(image)
        
        # Resize to target size
        image = image.resize(target_size, Image.Resampling.LANCZOS)
        
        return image
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None


def create_transforms(mean=0.5, std=1.0):
    """
    Create image transforms matching your MNIST setup.
    
    Args:
        mean (float): Normalization mean
        std (float): Normalization standard deviation
        
    Returns:
        torchvision.transforms.Compose: Transform pipeline
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((mean,), (std,))
    ])
    return transform


def create_label_transform():
    """
    Create label transform matching your MNIST setup.
    
    Returns:
        torchvision.transforms.Compose: Label transform pipeline
    """
    label_transform = transforms.Compose([
        lambda x: torch.LongTensor([x])
    ])
    return label_transform




class CustomImageDataset(Dataset):
    """
    Custom dataset class that exactly mimics torchvision.datasets.MNIST structure.
    
    This dataset replicates ALL MNIST properties including .data and .targets tensors.
    """
    
    def __init__(self, image_paths, labels, target_size=(28, 28), 
                 convert_grayscale=True, apply_aging=False,
                 transform=None, target_transform=None):
        """
        Initialize the dataset with MNIST-compatible structure.
        
        Args:
            image_paths (list): List of image file paths
            labels (list): List of integer labels corresponding to images
            target_size (tuple): Target size for resizing images
            convert_grayscale (bool): Whether to convert images to grayscale
            apply_aging (bool): Whether to apply aging effects
            transform (callable): Transform to apply to images
            target_transform (callable): Transform to apply to labels
        """
        self.image_paths = image_paths
        self.target_size = target_size
        self.convert_grayscale = convert_grayscale
        self.apply_aging = apply_aging
        self.transform = transform
        self.target_transform = target_transform
        
        # Validate that we have equal number of images and labels
        if len(image_paths) != len(labels):
            raise ValueError(f"Number of images ({len(image_paths)}) must match number of labels ({len(labels)})")
        
        # Create MNIST-compatible attributes
        self.targets = torch.tensor(labels, dtype=torch.long)  # Shape: [N]
        self._data = None  # Will be loaded when first accessed
        
        print(f"Initializing dataset with {len(image_paths)} images...")
    
    @property
    def data(self):
        """
        MNIST-compatible data property that returns all images as a tensor.
        Shape: [N, H, W] for grayscale or [N, H, W, C] for color
        Dtype: torch.uint8 (same as MNIST)
        
        Images are loaded and cached on first access.
        """
        if self._data is None:
            print("Loading all images into .data tensor (this may take a moment)...")
            self._load_all_images()
        return self._data
    
    def _load_all_images(self):
        """Load all images into the .data tensor."""
        all_images = []
        
        for i, image_path in enumerate(self.image_paths):
            # Load and preprocess image
            image = load_and_preprocess_image(
                image_path,
                target_size=self.target_size,
                convert_grayscale=self.convert_grayscale,
                apply_aging=self.apply_aging
            )
            
            # Handle failed loading
            if image is None:
                if self.convert_grayscale:
                    image = Image.new('L', self.target_size, 0)
                else:
                    image = Image.new('RGB', self.target_size, (0, 0, 0))
            
            # Convert to numpy array with uint8 dtype (like MNIST)
            img_array = np.array(image, dtype=np.uint8)
            all_images.append(img_array)
            
            # Progress indicator
            if (i + 1) % 100 == 0:
                print(f"Loaded {i + 1}/{len(self.image_paths)} images...")
        
        # Stack all images and convert to tensor
        all_images = np.stack(all_images, axis=0)
        self._data = torch.from_numpy(all_images)
        
        print(f"Loaded .data tensor with shape: {self._data.shape}, dtype: {self._data.dtype}")
    
    def __len__(self):
        """Return the total number of samples."""
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """
        Get a single sample from the dataset.
        
        Args:
            idx (int): Index of the sample to retrieve
            
        Returns:
            tuple: (image, label) where image is a tensor and label is a tensor
            
        Note: This applies transforms to the raw data, just like MNIST
        """
        # Get raw image from .data tensor (this will load all images if needed)
        raw_image = self.data[idx]  # Shape: [H, W] or [H, W, C]
        
        # Convert tensor back to PIL Image for transforms
        if raw_image.dim() == 2:  # Grayscale
            image = Image.fromarray(raw_image.numpy(), mode='L')
        else:  # Color
            image = Image.fromarray(raw_image.numpy(), mode='RGB')
        
        # Get label from targets tensor
        label = self.targets[idx].item()  # Convert to Python int
        
        # Apply transforms (same as MNIST)
        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label




def create_image_dataset(image_paths, labels, target_size=(28, 28),
                        convert_grayscale=True, apply_aging=False,
                        mean=0.5, std=1.0):
    """
    Create a complete image dataset ready for use with DataLoader.
    
    Args:
        image_paths (list): List of image file paths
        labels (list): List of integer labels for the images
        target_size (tuple): Target size for resizing images
        convert_grayscale (bool): Whether to convert images to grayscale
        apply_aging (bool): Whether to apply aging effects
        mean (float): Normalization mean
        std (float): Normalization standard deviation
        
    Returns:
        CustomImageDataset: Dataset ready for use with DataLoader
    """
    # Step 1: Create transforms
    transform = create_transforms(mean=mean, std=std)
    target_transform = create_label_transform()
    
    # Step 2: Create dataset
    dataset = CustomImageDataset(
        image_paths=image_paths,
        labels=labels,
        target_size=target_size,
        convert_grayscale=convert_grayscale,
        apply_aging=apply_aging,
        transform=transform,
        target_transform=target_transform
    )
    
    return dataset



In [ ]:
def collate_ae_dataset(samples):
    """
    The function collates sampels into a batch, and creates noisy samples if DENOISING is True
    for the denoising autoencoder.
    """
    xs = [s[0] for s in samples]
    ys = [s[1] for s in samples]
    #Extracts the first element (input data) from each sample into list xs.
    #Extracts the second element (labels or targets) from each sample into list ys.
    #This assumes each sample is a tuple or list with at least two elements.
    
    xs = torch.stack(xs)
    ys = torch.concat(ys)
    #torch.stack(xs) combines the list of input tensors into a single 
    #tensor along a new dimension (creating a batch dimension).
    #torch.concat(ys) concatenates the label tensors along 
    #the existing first dimension. This suggests the labels might have 
    #variable lengths or already include a batch-like dimension.

    add_noise = NOISE_RATE > 0.
    #Checks if noise should be added based on a global variable NOISE_RATE.
    #If NOISE_RATE is greater than 0, noise will be added to the inputs.
    
    if add_noise:
      sh = xs.shape
      noise_mask = torch.bernoulli(torch.full(sh, NOISE_RATE))  # 0 (keep) or 1 (replace with noise)
      #Gets the shape of the input tensor batch.
      #Creates a binary mask using Bernoulli sampling, where each element has NOISE_RATE probability of being 1 
      #(indicating where noise will be applied) and 1-NOISE_RATE probability of being 0.
            
      sp_noise = torch.bernoulli(torch.full(sh, 0.5))-0.5  # -1 or 1
      #Generates the actual noise values as either -0.5 or 0.5.
      #First creates a tensor of the same shape filled with 0.5, then applies Bernoulli 
      #sampling to get 0s or 1s.
      #Subtracts 0.5 to convert to -0.5 or 0.5 (this creates salt and pepper noise).
        
      xns = xs * (1-noise_mask) + sp_noise * noise_mask
      #Creates the noisy input xns by:
          #Keeping original values where the mask is 0: xs * (1-noise_mask)
          #Adding noise values where the mask is 1: sp_noise * noise_mask
          #The result is a tensor where some values are preserved 
          #from the original input and others are replaced with noise.
      
      # sp = sp_noise
    else:
       xns = xs
    #If no noise is to be added, the noisy input is the same as the original input.

    return xns.to(device), xs.to(device), ys.to(device)
    #Returns three tensors, all moved to the specified device (likely GPU):

    #xns: The inputs with noise added (or original inputs if no noise)
    #xs: The original clean inputs
    #ys: The labels or targets
    #
    #
    #This return structure is typical for denoising autoencoders, where you need 
    #both the noisy input (fed to the encoder) and the clean target 
    #(used to compute the reconstruction loss).
    #
    #This function is specifically designed for training denoising autoencoders, 
    #where the model learns to remove noise from corrupted inputs by trying 
    #to reconstruct the original clean data.

### Functions to Prepare and Execute Training

In [ ]:
def get_samples(valid_loader):
  # 1. get numpy array of all validation images:
  val_images_noisy = []
  val_images = []
  val_labels = []

  for batch_idx, (noisy_data, data, target) in enumerate(valid_loader):
      val_images_noisy.append(noisy_data.detach().cpu().numpy())
      val_images.append(data.detach().cpu().numpy())
      val_labels.append(target.detach().cpu().numpy())

  val_images_noisy = np.concatenate(val_images_noisy, axis=0)
  val_images = np.concatenate(val_images, axis=0)
  val_labels = np.concatenate(val_labels, axis=0)

  # 2. get numpy array of balanced validation samples for visualization:
  sample_images_noisy = []
  sample_images = []
  sample_labels = []
  single_el_idx = []  # indexes of single element per class for visualization

  n_class = np.max(val_labels) + 1
  # Determines the number of classes (for MNIST, this would be 10, representing digits 0-9).
  for class_idx in range(n_class):
    map_c = val_labels == class_idx

    ims_c_noisy = val_images_noisy[map_c]
    ims_c = val_images[map_c]
    print('class label:')
    print(class_idx)
    print('shape selected class')
    print(ims_c.shape)
    # For each class:
       # Creates a boolean mask map_c identifying all samples of the current class.
       # Extracts noisy and clean images for just this class.
      

    samples_idx = np.random.choice(len(ims_c), N_SAMPLE, replace=False)

    ims_c_noisy_samples = ims_c_noisy[samples_idx]
    ims_c_samples = ims_c[samples_idx]
    # Randomly selects N_SAMPLE images from the current class.
    # replace=False ensures no duplicates are selected.
    # Extracts both noisy and clean versions of these sampled images.
      

    sample_images_noisy.append(ims_c_noisy_samples)
    sample_images.append(ims_c_samples)

    sample_labels.append([class_idx]*N_SAMPLE)

    # Adds the sampled noisy images, clean images, and labels to their respective lists.
    # Creates an array of N_SAMPLE repeated labels for this class.

    start_idx = N_SAMPLE*class_idx
    single_el_idx.extend([start_idx + i for i in range(min(N_VIS_SAMPLE, N_SAMPLE))])
    # Calculates the indices for the first N_VIS_SAMPLE elements of this class in the final concatenated array.
    # These indices will be used to extract a smaller subset for visualization.

    
  sample_images_noisy = np.concatenate(sample_images_noisy, axis=0)
  sample_images = np.concatenate(sample_images, axis=0)
  sample_labels = np.concatenate(sample_labels, axis=0)
  single_el_idx = np.array(single_el_idx)
  #Combines all class samples into single arrays.
  #Converts the index list to a NumPy array.

  samples = {
      'images_noisy': sample_images_noisy,
      'images': sample_images,
      'labels': sample_labels,
      'single_el_idx': single_el_idx

  }
  return samples
# Creates and returns a dictionary with all collected samples.


# This function ensures we have:
# 
# A balanced number of samples for each class (equal representation)
# Both noisy and clean versions of each image
# A mapping between the noisy and clean versions
# A subset of indices for visualization purposes
# This is particularly useful for creating visualizations that show how the model behaves across different classes, or for comparing reconstruction quality across digits.



In [ ]:
def to_np_showable(pt_img):
  np_im = pt_img.detach().cpu().numpy()
  if len(np_im.shape) == 4:
    np_im = np_im[0]

  if np_im.shape[0] > 3:
    np_im = np_im[-3:]

  return (eo.rearrange(np_im, 'c h w -> h w c')/2+.5).clip(0., 1.)

#This function converts a PyTorch tensor image to a NumPy array suitable for visualization.
#pt_img.detach().cpu().numpy() - Detaches the tensor from the computation graph, moves it to CPU if it's on GPU, and converts it to a NumPy array.
#if len(np_im.shape) == 4: - Checks if the image has a batch dimension (shape: [batch, channels, height, width]).
#np_im = np_im[0] - If there's a batch dimension, takes only the first image in the batch.
#if np_im.shape[0] > 3: - Checks if there are more than 3 channels.
#np_im = np_im[-3:] - If there are more than 3 channels, keeps only the last 3 channels (useful for handling multi-channel data).
#eo.rearrange(np_im, 'c h w -> h w c') - Uses the einops library to rearrange the tensor from PyTorch's [channels, height, width] format to matplotlib's [height, width, channels] format.
#/2+.5 - Applies normalization assuming the image data is in the range [-1, 1], converting it to [0, 1].
#.clip(0., 1.) - Ensures all values are within the [0, 1] range, clamping any values outside this range.

def plot_im(im, is_torch=True):
  plt.imshow(to_np_showable(im) if is_torch else im, cmap='gray')
  plt.show()
  plt.close()

#This function plots a single image.
#is_torch=True - Default parameter indicating whether the input is a PyTorch tensor.
#to_np_showable(im) if is_torch else im - Converts the image to a NumPy array if it's a PyTorch tensor, otherwise uses it directly.
#plt.imshow(..., cmap='gray') - Displays the image using matplotlib with a grayscale colormap.
#plt.show() - Renders the plot.
#plt.close() - Closes the figure to free up memory.

def plot_im_samples(ds, n=5, is_torch=False):
  fig, axs = plt.subplots(1, n, figsize=(16, n))
  for i, image in enumerate(ds[:n]):
      axs[i].imshow(to_np_showable(image) if is_torch else image, cmap='gray')
      axs[i].set_axis_off()
  plt.show()
  plt.close()

#This function plots multiple images from a dataset in a row.
#ds - The dataset or collection of images to sample from.
#n=5 - Default number of images to display.
#is_torch=False - Default parameter indicating whether the inputs are PyTorch tensors.
#plt.subplots(1, n, figsize=(16, n)) - Creates a figure with a single row of n subplots, with a width of 16 inches and height of n inches.
#The loop iterates through the first n images in the dataset:
#
#axs[i].imshow(...) - Displays each image in its corresponding subplot.
#axs[i].set_axis_off() - Removes axis labels and ticks for cleaner visualization.
#
#
#plt.show() - Renders the entire plot with all images.
#plt.close() - Closes the figure to free up memory.

In [ ]:
# get mean and std of an array with numpy:
def get_mean_std(x):
    x_mean = np.mean(x)
    x_std = np.std(x)
    return x_mean, x_std

# get min and max of an array with numpy:
def get_min_max(x):
    x_min = np.min(x)
    x_max = np.max(x)
    return x_min, x_max

def is_iterable(obj):
    try:
        iter(obj)
    except Exception:
        return False
    else:
        return True

#This function checks if an object is iterable (can be looped over).
#It uses a try-except block to attempt to call iter(obj), which will succeed only if obj is iterable.
#If calling iter(obj) raises any exception, the function returns False.
#If no exception occurs, the function returns True.

def type_len(obj):
    t = type(obj)
    if is_iterable(obj):
        sfx = f', shape: {obj.shape}' if t == np.ndarray else ''
        print(f'type: {t}, len: {len(obj)}{sfx}')
    else:
        print(f'type: {t}, len: {len(obj)}')

#This is a utility function for debugging that prints information about an object.
#t = type(obj) - Gets the type of the provided object.
#It checks if the object is iterable using the is_iterable function defined earlier.
#If the object is iterable:
#
#It checks if the object is a NumPy array (t == np.ndarray).
#If it's a NumPy array, it adds shape information to the output string.
#It prints the type and length of the object, along with shape information if applicable.
#
#
#If the object is not iterable, it still attempts to print the type and length (though this might raise an error if len() isn't applicable to the object).
#
#Note: There seems to be an issue with the type_len function - it tries to call len() on non-iterable objects in the else clause, 
#which would typically cause an error. This might be a bug in the code.


In [ ]:
# merging 2d matrix of images in 1 image
def mosaic(mtr_of_ims):
  ny = len(mtr_of_ims)
  assert(ny != 0)
  #Gets the number of rows in the matrix and asserts that it's not empty.

  nx = len(mtr_of_ims[0])
  assert(nx != 0)
  #Gets the number of columns in the first row and asserts that it's not empty.

  im_sh = mtr_of_ims[0][0].shape

  assert (2 <= len(im_sh) <= 3)
  #Gets the shape of the first image in the matrix.
  #Verifies that the image is either 2D (grayscale) or 3D (with channels).
    
  multichannel = len(im_sh) == 3

  if multichannel:
    h, w, c = im_sh
  else:
    h, w = im_sh
  #Determines if the images have multiple channels.
  #If multichannel, unpacks height, width, and channels. Otherwise, just height and width.

  h_c = h * ny + 1 * (ny-1)
  w_c = w * nx + 1 * (nx-1)
  #Calculates the total height and width of the canvas.
  #Adds 1 pixel spacing between images (both horizontally and vertically).

  canv_sh = (h_c, w_c, c) if multichannel else (h_c, w_c)
  canvas = np.ones(shape=canv_sh, dtype=np.float32)*0.5
  #Defines the shape of the canvas based on whether images are multichannel.
  #Creates a canvas filled with gray (0.5) values, assuming image values are in [0,1] range.

  for iy, row in enumerate(mtr_of_ims):
    y_ofs = iy * (h + 1)
    #Loops through each row of images.
    #Calculates the vertical offset for the current row.
    for ix, im in enumerate(row):
      x_ofs = ix * (w + 1)
      #Loops through each image in the current row.
      #Calculates the horizontal offset for the current image.
      canvas[y_ofs:y_ofs + h, x_ofs:x_ofs + w] = im
      #Copies the current image to the appropriate position in the canvas.
      #This uses NumPy's array slicing to place the image at the correct location.
  return canvas

In [ ]:
from collections import defaultdict
import random

def split_train_validation(image_paths, labels, train_ratio=0.8, random_seed=42):
    """
    Split image paths into training and validation sets with balanced class distribution.
    
    Args:
        image_paths (list): List of image file paths
        labels (list): List of class labels corresponding to image paths
        train_ratio (float): Proportion for training set (default 0.8)
        random_seed (int): Random seed for reproducibility
        
    Returns:
        tuple: (train_paths, train_labels, val_paths, val_labels)
    """
    random.seed(random_seed)
    
    # Group paths by class
    class_groups = defaultdict(list)
    for path, label in zip(image_paths, labels):
        class_groups[label].append(path)
    
    train_paths, train_labels = [], []
    val_paths, val_labels = [], []
    
    # Split each class maintaining proportion
    for class_label, paths in class_groups.items():
        # Shuffle paths for this class
        shuffled_paths = paths.copy()
        random.shuffle(shuffled_paths)
        
        # Calculate split point
        n_train = int(len(shuffled_paths) * train_ratio)
        
        # Split paths
        class_train_paths = shuffled_paths[:n_train]
        class_val_paths = shuffled_paths[n_train:]
        
        # Add to final lists
        train_paths.extend(class_train_paths)
        train_labels.extend([class_label] * len(class_train_paths))
        val_paths.extend(class_val_paths)
        val_labels.extend([class_label] * len(class_val_paths))
    
    # Final shuffle to mix classes
    combined_train = list(zip(train_paths, train_labels))
    combined_val = list(zip(val_paths, val_labels))
    random.shuffle(combined_train)
    random.shuffle(combined_val)
    
    train_paths, train_labels = zip(*combined_train) if combined_train else ([], [])
    val_paths, val_labels = zip(*combined_val) if combined_val else ([], [])
    
    print(f"Training set: {len(train_paths)} images")
    print(f"Validation set: {len(val_paths)} images")
    
    return list(train_paths), list(train_labels), list(val_paths), list(val_labels)

### Functions To Analyse Results

In [ ]:
def plot_hist(history, logscale=True):
    """
    plot training loss
    """

    loss = history['loss']
    v_loss = history['val_loss']
    epochs = history['epoch']

    # This function visualizes training history (loss over epochs).
    # Extracts training loss, validation loss, and epoch numbers from the history dictionary.

    
    plot = plt.semilogy if logscale else plt.plot
    # Cleverly chooses between logarithmic scale (plt.semilogy) or linear scale (plt.plot) based on the logscale parameter.
    # Default is logarithmic scale, which is often better for visualizing loss curves as they typically decrease exponentially.
    
    plot(epochs, loss, label='training');
    plot(epochs, v_loss, label='validation');
    # Plots both training and validation loss curves using the selected plotting function.
    # Labels each curve for the legend.
    
    plt.legend()
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()
    plt.close()
    # Adds a legend, axis labels, displays the plot, and then closes the figure.



def plot_samples(sample_history, samples, epoch_stride=5, fig_scale=1):
    """
    Plots input, noisy samples (for DAE) and reconstruction.
    Each `epoch_stride`-th epoch
    """
    # This function visualizes sample reconstructions over training epochs.
    # Shows how the model's reconstruction capability improves over time.

    single_el_idx = samples['single_el_idx']
    images_noisy = samples['images_noisy'][single_el_idx, 0]
    images = samples['images'][single_el_idx, 0]
    # Extracts indices for selected samples to visualize.
    # Gets the noisy input images and the original clean images for these samples.
    # The , 0 indexing suggests selecting the first channel of each image.

    last_epoch = np.max(list(sample_history.keys()))
    # Determines the last epoch number in the history data.

    for epoch_idx, hist_el in sample_history.items():
      if epoch_idx % epoch_stride != 0 and epoch_idx != last_epoch:
        continue
    # Iterates through each epoch's results in the history.
    # Uses epoch_stride to select only every nth epoch (to avoid too many visualizations).
    # Always includes the last epoch regardless of the stride.

      samples_arr = [images_noisy, hist_el['y'][single_el_idx, 0], images]
    # Creates an array of three sets of images to visualize side by side:
       # The noisy input images
       # The model's reconstructions for the current epoch
       # The original clean images (ground truth)

      ny = len(samples_arr)
      nx = len(samples_arr[0])

      plt.figure(figsize=(fig_scale*nx, fig_scale*ny))
      # Calculates the dimensions of the visualization grid.
      # Creates a figure with size proportional to the number of samples.

        
      m = mosaic(samples_arr)
      # Uses the previously defined mosaic function to create a grid of all images.

      plt.title(f'after epoch {int(epoch_idx)}')
      plt.imshow(m, cmap='gray', vmin=-.5, vmax=.5)
      # Adds a title showing which epoch this visualization represents.
      # Displays the mosaic with a grayscale colormap and fixed value range.
      # The vmin=-.5, vmax=.5 matches the normalized data range we've seen before.

        
      plt.tight_layout(pad=0.1, h_pad=0, w_pad=0)
      plt.show()
      plt.close()
      # Ensures proper spacing in the figure.
      # Displays the figure and then closes it to free memory.

# This function creates a powerful visualization showing the progression of the model's reconstruction ability across epochs. Each visualization has three rows:
# 
# The noisy inputs
# The model's reconstructions
# The original clean images (targets)
# 
# This makes it easy to see how the model gradually learns to denoise and reconstruct the images over the course of training.

In [ ]:
# These are utility functions for working with trained models at different stages of training. Let me break them down:

def run_on_trained(model, root_dir, run_fn, ep=None, model_filename=None):
    """
    Helper function to excecute any function on model in state after `ep` training epoch
    """
    # This function loads a model checkpoint and runs a specified function on it.
    # Parameters:
    # 
    # model: The neural network model instance
    # root_dir: Directory containing saved model checkpoints
    # run_fn: The function to run on the loaded model
    # ep: Specific epoch to load (optional)
    # model_filename: Specific checkpoint file to load (optional)

    if model_filename is None:
        if ep is not None:
            model_filename = root_dir/f'model_{ep:03d}.pth'
        else:
            model_filename = sorted(list(root_dir.glob('*.pth')))[-1]  # last model state
    # Determines which model checkpoint file to load:
    # 
    # If a specific filename is provided, use that (in this case this code block would be skipped)
    # If an epoch number is provided, construct the filename using a pattern
    # If neither is provided, use the last checkpoint file (by alphabetical sorting)
    # The code uses pathlib's Path objects for file handling (using / for path joining)

    
    model_dict = torch.load(model_filename,weights_only=False)

    model.load_state_dict(model_dict['model_state_dict'])

    # Loads the saved model state from the specified file
    # The weights_only=False parameter indicates to load the full state dictionary (not just weights)
    # Restores the model parameters from the saved state dictionary
    

    run_fn(model)
    # Calls the provided function on the loaded model

def run_on_all_training_history(model, root_dir, run_fn, n_ep=None):
    """
    Helper function to excecute any function on model state after each of the training epochs
    """
    # This function runs a specified function on multiple model checkpoints from different training epochs.
    # Parameters:
    # 
    # model: The neural network model instance
    # root_dir: Directory containing saved model checkpoints
    # run_fn: The function to run on each loaded model state
    # n_ep: Specific number of epochs to process (optional)
    
    if n_ep is not None:
        for ep in range(n_ep):
            print(f'running on epoch {ep+1}/{n_ep}...')
            run_on_trained(model, root_dir, run_fn, ep=ep)
    # If a specific number of epochs is provided:
    # 
    # Iterates through each epoch from 0 to n_ep-1
    # Prints progress information
    # Calls run_on_trained for each epoch
    
    else:
        for model_filename in sorted(root_dir.glob('*.pth')):
            print(f'running on checkpoint {model_filename}...')
            run_on_trained(model, root_dir, run_fn, model_filename=model_filename)

    # If no specific number of epochs is provided:
    # 
    # Finds all .pth files in the root directory
    # Sorts them (presumably by name, which would be by epoch if using the naming pattern)
    # Processes each checkpoint file in order
    
    print(f'done')

    # Prints a completion message when all checkpoints have been processed
    # 
    # These utility functions make it easy to:
    # 
    # Analyze a model at a specific point in its training history
    # Run the same analysis across multiple stages of training
    # Visualize or evaluate how the model's behavior changes over the course of training
    # 
    # They're particularly useful for post-training analysis, debugging, and creating visualizations of model evolution.

### Set Globals

In [ ]:
# Set globals (your existing code)
BATCH_SIZE = 32

device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
print("Using device:", device)


N_SAMPLE = 4*32
N_VIS_SAMPLE = 2
CODE_SIZE = 120
NOISE_RATE = 0.1
MODEL_NAME = 'cdae_model'
SIDE_LENGTH = 320

target_size = (SIDE_LENGTH, SIDE_LENGTH)

### Define file paths: 

In [ ]:

project_path = Path.cwd()
root_path = (project_path / '..').resolve()

# Define paths
image_dir = root_path/'visual_genome_proc_data'  # Replace with your directory containing images

### Get information about image size: 

In [ ]:
results = analyze_image_sizes(image_dir, extensions=('.jpg', '.jpeg', '.tif', '.tiff'))
results

### Load meta data about images: 

In [ ]:
meta_data = pd.read_csv(image_dir/'labels.csv')
meta_data.head()

In [ ]:
image_paths = load_image_paths(image_dir)
image_paths[0:3]

In [ ]:
image_paths = list(meta_data.file_paths)
#labels = list(meta_data.mountains)
labels = list(meta_data.buildings)

### Define Basic Autoencoder Class

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size, code_size):
        self.input_size = list(input_size)  # shape of data sample
        self.flat_data_size = np.prod(self.input_size)
        self.hidden_size = 128

        self.code_size = code_size  # code size

        super(AutoEncoder, self).__init__()
        #Creates an autoencoder neural network that inherits from PyTorch's nn.Module.
        #Takes two parameters:
        #
        #input_size: The shape of input data (e.g., [1, 28, 28] for MNIST)
        #code_size: The dimension of the encoded representation (bottleneck)
        #
        #
        #Calculates the flattened input size by multiplying all dimensions.
        #Sets an intermediate hidden layer size of 128 neurons.
        #Calls the parent class initializer.

        
        self.encoder = nn.Sequential(
            nn.Flatten(),

            nn.Linear(self.flat_data_size, self.hidden_size),
            nn.ReLU(),

            nn.Linear(self.hidden_size, self.code_size),
            nn.Sigmoid(),
        )
        #Defines the encoder network as a sequence of operations:
            #
            #Flattens the input (e.g., converts a 2D image to 1D)
            #Linear layer mapping from input size to hidden size
            #ReLU activation
            #Linear layer mapping from hidden size to code size
            #Sigmoid activation (constrains the encoded values to [0, 1])
        
        self.decoder = nn.Sequential(
            nn.Linear(self.code_size, self.hidden_size),
            nn.ReLU(),

            nn.Linear(self.hidden_size, self.flat_data_size),
            nn.Tanh(),  # Think: why tanh?

            nn.Unflatten(1, self.input_size),
        )
        #Defines the decoder network:
            #Linear layer from code size to hidden size
            #ReLU activation
            #Linear layer from hidden size back to the flattened input size
            #Tanh activation (outputs values in [-1, 1], matching the normalized input range)
            #Unflattens the output back to the original input shape

#Regarding "why tanh?": Tanh is used because the input images were normalized to approximately [-0.5, 0.5] 
    #range (using m=0.5, s=1.0). Tanh outputs values in the range [-1, 1], 
    #which after scaling by 1.1 in the decode method closely matches the input data range.

    def forward(self, x, return_z=False):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return (decoded, encoded) if return_z else decoded
    # The forward pass:
        #Encodes the input
        #Decodes the encoded representation
        #If return_z=True, returns both the reconstruction and the encoded values
        #Otherwise, just returns the reconstruction
        

    def encode(self, x):
        return self.encoder(x)

    def decode(self, z):
        return self.decoder(z)*1.1
# Helper methods to encode and decode separately
# Note the multiplication by 1.1 in the decode method, 
    # which slightly amplifies the output range to better match the input data distribution

        

    def get_n_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)
    # Utility method to count the total number of trainable parameters in the model


def eval_on_samples(ae_model, epoch, samples):
    # this is called on end of each training epoch
    xns = samples['images_noisy']
    xns = torch.tensor(xns, dtype=torch.float32).to(device)
    #labels = samples['labels']

# Function to evaluate the autoencoder on sample data after each epoch
# Takes the model, current epoch number, and samples dictionary
# Extracts noisy images from the samples and converts them to a PyTorch tensor on the target device
# The labels are extracted but commented out (not used)

    with torch.no_grad():
        yz = ae_model(xns, return_z=True)
        yz = [el.detach().cpu().numpy() for el in yz]

        y = yz[0]
        z = yz[1:]
    # Uses torch.no_grad() to disable gradient calculation (for efficiency during evaluation)
    # Gets both reconstructions and encodings (i.e. latent space!) by calling the model with return_z=True
    # Converts all outputs to NumPy arrays
    # Separates the reconstructions y and encodings z

    res = {'z': z, 'y': y, 'epoch': epoch}
    return res

# Creates and returns a dictionary containing:

# z: The encoded representations
# y: The reconstructed images
# epoch: The current epoch number
# 

# This evaluation function captures the model's performance at each epoch, allowing for tracking reconstruction quality and analyzing the learned representations over time.

### Split data in training set and validation set, execute workflow: 

In [ ]:
train_paths, train_labels, val_paths, val_labels = split_train_validation(image_paths, labels, train_ratio=0.8, random_seed=42)


### Load Data

In [ ]:
# Step 1: Load image paths
# already done

# Step 2: Create your labels list (matching image_paths order)
# already done

# Step 3: Create dataset (equivalent to train_dataset)
train_dataset = create_image_dataset(
    image_paths=train_paths,
    labels=train_labels,
    target_size=target_size,
    convert_grayscale=True,
    apply_aging=False,
    mean=0.5,
    std=1.0
)

In [ ]:


# Step 4: Create DataLoader with your original collate function
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_ae_dataset,  # Your original function using globals
    drop_last=True
)


In [ ]:
valid_dataset = create_image_dataset(
    image_paths=val_paths,
    labels=val_labels,
    target_size=target_size,
    convert_grayscale=True,
    apply_aging=False,
    mean=0.5,
    std=1.0
)

In [ ]:
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_ae_dataset,  # Your original function using globals
    drop_last=True
)

### Define Convolutional Autoencoder (inherits some properties from basic autoencoder)

In [ ]:
final_spatial_size = 320 // (2**6)

#in_size = target_size
#in_size

In [ ]:
class ConvolutionalAutoEncoder(AutoEncoder):
    def __init__(self, input_size, code_size):
        self.input_size = list(input_size)
        
        # For 320x320 input with 6 stride=2 layers: 320÷64 = 5x5 final spatial size
        # With hidden_fs=64: final feature map is 64×5×5 = 1600 values
        self.hidden_size = 64 * 5 * 5  # = 1600 (not 128!)
        
        self.code_size = code_size
        
        super(ConvolutionalAutoEncoder, self).__init__(input_size, code_size)
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, stride=1), nn.LeakyReLU(negative_slope=0.3),      # 320×320×8
            nn.Conv2d(8, 8, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),      # 160×160×8
            nn.Conv2d(8, 16, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),     # 80×80×16
            nn.Conv2d(16, 16, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),    # 40×40×16
            nn.Conv2d(16, 32, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),    # 20×20×32
            nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),    # 10×10×32
            nn.Conv2d(32, 64, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),    # 5×5×64
            
            nn.Flatten(),  # 5×5×64 = 1600 values
            
            nn.Linear(1600, 200), nn.LeakyReLU(negative_slope=0.3),  # 1600 → 200
            nn.Linear(200, code_size),  # 200 → 70
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(code_size, 1600), nn.LeakyReLU(negative_slope=0.3),  # 70 → 1600
            
            nn.Unflatten(1, (64, 5, 5)),  # 1600 → 64×5×5
            
            nn.ConvTranspose2d(64, 32, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 5→10
            nn.ConvTranspose2d(32, 32, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 10→20
            nn.ConvTranspose2d(32, 16, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 20→40
            nn.ConvTranspose2d(16, 16, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 40→80
            nn.ConvTranspose2d(16, 8, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),   # 80→160
            nn.ConvTranspose2d(8, 8, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),    # 160→320
            
            nn.Conv2d(8, 1, 3, padding=1, stride=1), nn.Tanh(),  # Final output: 320×320×1
        )

    def decode(self, z):
        reconstruction = self.decoder(z)
        return reconstruction  # No cropping needed with correct dimensions

In [ ]:
in_size = target_size
model = ConvolutionalAutoEncoder(input_size=in_size, code_size=CODE_SIZE).to(device)

#samples = get_samples(valid_loader)
samples = get_samples(train_loader)

In [ ]:
xns = torch.tensor(samples['images_noisy']).to(device)
print(xns.shape)
zs = model.encode(xns)
ys = model(xns)
print(zs.shape)
print(ys.shape)

In [ ]:
# train the autoencoder model, for N_EPOCHS epochs,
# save history of loss values for training and validation sets,
# history of validation samples evolution, and model weights history,


# This code implements the complete training loop for the autoencoder. 
# Let me break it down:

#N_EPOCHS = 60
N_EPOCHS = 6
LR = 0.0009
# Sets the number of training epochs to 50
# Sets the learning rate for the Adam optimizer to 0.0009

model_root = pl.Path(MODEL_NAME)
model_root.mkdir(exist_ok=True)
# Creates a directory path for saving model checkpoints using the MODEL_NAME ('ae_model')
# Makes sure the directory exists (creates it if it doesn't)

optimizer = optim.Adam(model.parameters(), lr=LR)
# Creates an Adam optimizer to update the model parameters
# Adam is an adaptive learning rate optimization algorithm well-suited for deep learning

# implement loss explicitly
loss = nn.MSELoss()
# Defines the loss function as Mean Squared Error (MSE)
# This measures the average squared difference between the reconstructed and target images

# train the model
history = {'loss': [], 'val_loss': [], 'epoch': []}
sample_history = {}
# Creates dictionaries to store training metrics and sample reconstructions
# history tracks training and validation losses across epochs
# sample_history will store sample reconstruction results at each epoch

pbar = tqdm.tqdm(range(N_EPOCHS), postfix=f'epoch 0/{N_EPOCHS}')
# Creates a progress bar for tracking the training process
# Will show the current epoch and update metrics during training

for epoch_idx in pbar:
# Starts the main training loop that runs for N_EPOCHS iterations
    epoch_loss = 0
    model.train()
    # Initializes the epoch loss accumulator
    # Sets the model to training mode (enables dropout, batch normalization updates, etc.)
    
    for batch_idx, (noisy_data, data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(noisy_data)
        loss_value = loss(output, data)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.detach().cpu().item()
    # Iterates through all batches in the training dataset
    # For each batch:
    # 
        # Zeros out previous gradients
        # Passes the noisy input through the model
        # Calculates the MSE loss between the reconstruction and clean data
        # Computes gradients via backpropagation
        # Updates model parameters using the optimizer
        # Accumulates the loss value for epoch-level reporting
    
    epoch_loss /= len(train_loader)
    history['loss'].append(epoch_loss)
    history['epoch'].append(epoch_idx)
    # update progress bar

    # Calculates the average loss for the epoch
    # Records the loss and epoch number in the history

    # evaluate on validation set
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch_idx, (noisy_data, data, target) in enumerate(valid_loader):
            output = model(noisy_data)
            loss_value = loss(output, data)
            val_loss += loss_value.detach().cpu().item()
        val_loss /= len(valid_loader)
        history['val_loss'].append(val_loss)

    # Sets the model to evaluation mode (disables dropout, etc.)
    # Disables gradient calculation for efficiency
    # Computes the validation loss on the entire validation set
    # Records the average validation loss in the history
    
    pbar.set_postfix({'epoch': f'{epoch_idx+1}/{N_EPOCHS}', 'loss':f'{epoch_loss:.4f}', 'val_loss':f'{val_loss:.4f}'})
    # evaluate on samples
    # Updates the progress bar with current epoch, training loss, and validation loss
    
    sample_res = eval_on_samples(model, epoch_idx, samples=samples)
    # This saves the reconstructions and the latent space thanks to
    # the eval_on_samples function where in the application of the 
    # model to the evaluation data the return_z parameter is set 
    # to true: 
    # with torch.no_grad():
    #     yz = ae_model(xns, return_z=True)
    #     yz = [el.detach().cpu().numpy() for el in yz]
# 
    #     y = yz[0]
    #     z = yz[1:]
    
    # The output of eval_on_samples looks like this: 

    # sample_res = {'z': z, 'y': y, 'epoch': epoch}

    
    sample_history[epoch_idx] = sample_res
    # Evaluates the model on the sample images
    # Stores reconstructions for later visualization

    # save model weights
    torch.save({
                'epoch': epoch_idx,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss
                }, model_root/f'model_{epoch_idx:03d}.pth')

    # Saves a checkpoint of the model at each epoch
    # The checkpoint includes:
    # 
    # Current epoch number
    # Model parameters
    # Optimizer state (allows resuming training)
    # Loss function
    # 
    # 
    # Uses a formatted filename with padded epoch number (e.g., 'model_001.pth')
# 
# This is a complete training pipeline that not only trains the model 
# but also tracks metrics, evaluates on validation data, 
# and creates visualizations to monitor progress - 
# all while saving checkpoints for later analysis or resuming training.
    

In [ ]:
plot_hist(history)

In [ ]:
sample_history[59].keys()

In [ ]:
type(sample_history[59]['y'])

In [ ]:
sample_history[59]['y'].shape

In [ ]:
sample_history[59]['z'][0].shape

In [ ]:
plot_samples(sample_history, samples=samples, epoch_stride=5, fig_scale=1)

In [ ]:
%%capture
# Jupyter notebook cell magic to suppress output (useful for code that might produce verbose output)

single_el_idx = samples['single_el_idx']
images_noisy = samples['images_noisy'][single_el_idx, 0]
images = samples['images'][single_el_idx, 0]
# Extracts the indices for the selected sample images
# Gets the noisy input images and original clean images for these indices
# The , 0 selects the first channel (since these are grayscale images)

smpl_ims = []
for epoch_idx, hist_el in sample_history.items():
    samples_arr = [images_noisy, hist_el['y'][single_el_idx, 0], images]
    smpl_ims.append(samples_arr)
# Creates a list to store image arrays for each epoch
# For each epoch in the training history:
    # 
    # Creates an array containing [noisy inputs, reconstructions, original images]
    # Adds this array to the list

ny, nx = len(smpl_ims[0]), len(smpl_ims[0][0])
# Determines the number of rows (3: noisy, reconstructed, original) and columns (number of samples)

plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above animations use JavaScript
# Sets matplotlib to use JavaScript for HTML animations in Jupyter

s=1
fig = plt.figure(figsize=(s*nx, s*ny))
# Creates a figure with size proportional to the number of images

m = mosaic(smpl_ims[0])

ttl = plt.title(f'after epoch {int(0)}')
# plot 0th epoch - 0th frame
imsh = plt.imshow(m, cmap='gray', vmin=-0.5, vmax=0.5)
# Creates the initial frame of the animation using the first epoch's images
# Uses the mosaic function to arrange the images in a grid
# Sets grayscale colormap with value range [-0.5, 0.5]

# this function will be called to render each of the frames
def animate(i):
    m = mosaic(smpl_ims[i])
    imsh.set_data(m)

    ttl.set_text(f'after epoch {i}')

    return imsh

# Defines a function to update the plot for each frame of the animation
# Creates a mosaic of images for the current epoch
# Updates the image data and title text
# Returns the updated image object

# create animation
ani = animation.FuncAnimation(fig, animate, frames=len(smpl_ims))

# Creates an animation that calls the animate function for each epoch
# The result is a dynamic visualization showing how reconstructions evolve throughout training

# This animation provides an intuitive way to observe the autoencoder's learning 
# progress, allowing you to see how the model gradually improves at reconstructing 
# the original images from the noisy inputs.



In [ ]:
# display animation
ani

In [ ]:
%%capture
plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
fig = plt.figure(figsize=(8,8))
# Suppresses output with %%capture
# Sets matplotlib to use JavaScript for HTML animations
# Creates a square figure with size 8×8 inches

labels = samples['labels']
epochs = sorted(sample_history.keys())
z_res = [sample_history[ep]['z'][0] for ep in epochs]
# Gets the digit labels from the samples dictionary
# Creates a sorted list of all epoch numbers
# Extracts the latent space representations from each epoch

scat = plt.scatter(z_res[0][:,14], z_res[0][:,6], c=labels, cmap=cm.rainbow)
# Creates a scatter plot using the first two dimensions of the latent space from the first epoch
# Colors the points according to their digit labels (0-9)
# Uses the rainbow colormap to distinguish between different digits

plt.xlim(-60.1, 60.1)
plt.ylim(-60.1, 60.1)

ax = plt.gca()
legend1 = ax.legend(*scat.legend_elements(), title="digits")
ax.add_artist(legend1)
ax.set_aspect('equal')
ttl = plt.title(f'after epoch {0}')
# Sets fixed axis limits for consistent visualization across frames
# Gets the current axis
# Adds a legend showing the mapping between colors and digit classes
# Sets the aspect ratio to equal so circles appear as circles
# Adds a title showing the current epoch

def animate(i):
    z = z_res[i]
    scat.set_offsets(z)
    ttl.set_text(f'after epoch {i}')
    return scat

# Defines a function to update the plot for each animation frame
# Updates the scatter plot with the latent representations from the current epoch
# Updates the title text with the current epoch number
# Returns the updated scatter plot object

ani = animation.FuncAnimation(fig, animate, frames=len(z_res))

# Creates an animation that runs through all epochs

# This animation shows how the model progressively learns to organize 
# the latent space, with points representing the same digit class gradually 
# clustering together. It's a powerful visualization that helps understand 
# how the autoencoder is learning meaningful representations and 
# separating different classes in the latent space, even though it's 
# trained in an unsupervised manner without using the labels for training.


In [ ]:
ani

In [ ]:
latent_space = sample_history[59]['z'][0]

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity= 1, init="pca", learning_rate="auto", random_state=42)
X_valid_2D = tsne.fit_transform(latent_space)

In [ ]:
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], s=10, cmap="tab10")
plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Example array with shape (10, 2)
# Replace this with your actual data
data = X_valid_2D.copy()


inertia = []
k_range = range(1, 20)  # Test from 1 to 9 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(data)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 6))
plt.plot(k_range, inertia, 'o-', color='blue')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Specify the number of clusters (k)
# You can adjust this based on your needs
k = 12

# Initialize and fit the k-means model
kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
kmeans.fit(data)

# Get cluster assignments for each data point
labels = kmeans.labels_

# Get the coordinates of the cluster centers
centers = kmeans.cluster_centers_

# Print results
print("Cluster assignments:", labels)
print("Cluster centers:", centers)

### Link cluster data from latent space to the respective sample images:

In [ ]:
image_indices = list(range(len(labels)))

cluster_data = pd.DataFrame({'image_index': image_indices,
                            'cluster_label': labels, 
                             'class_label': samples['labels'], 
                            'feature_1': data[:, 0], 
                             'feature_2': data[:, 1]})
cluster_data.head()

In [ ]:
print(latent_space.shape)
print(data.shape)
print(cluster_data.shape)
print(sample_history[59]['y'].shape)
print(sample_history[59]['z'][0].shape)

In [ ]:
samples_indexed = {}
for idx in cluster_data.image_index:
    image = samples['images'][idx][0]
    #print(image.shape)
    samples_indexed[idx] = image
    

In [ ]:
sample_history[59]['y'][1][0].shape

In [ ]:
reconstructions_indexed = {}
for idx in cluster_data.image_index:
    reconstruction = sample_history[59]['y'][idx][0]
    reconstructions_indexed[idx] = reconstruction


In [ ]:
# Visualize the clusters
plt.figure(figsize=(8, 6))
scatter = plt.scatter(cluster_data.feature_1, cluster_data.feature_2, c=cluster_data.cluster_label, cmap='viridis', s=100, alpha=0.7)

# Add cluster labels at cluster centers
unique_clusters = np.unique(cluster_data.cluster_label)
for cluster_id in unique_clusters:
    # Find points belonging to this cluster
    cluster_points = cluster_data[cluster_data.cluster_label == cluster_id]
    
    # Calculate centroid of this cluster
    center_x = cluster_points.feature_1.mean()
    center_y = cluster_points.feature_2.mean()
    
    # Add text label at the center
    plt.text(center_x, center_y, str(int(cluster_id)), 
             fontsize=12, fontweight='bold', ha='center', va='center',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.title('K-means Clustering Results')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Add colorbar to show cluster mapping
plt.colorbar(scatter, label='Cluster ID')

plt.grid(True, alpha=0.3)
plt.show()

# Calculate the silhouette score to evaluate the clustering quality
from sklearn.metrics import silhouette_score
if k > 1 and len(np.unique(labels)) > 1:  # Ensure there are at least 2 clusters with data
    score = silhouette_score(data, labels)
    print(f"Silhouette Score: {score:.3f}")

In [ ]:
cluster_data.shape

In [ ]:
def plot_images_from_cluster(cluster_data, cluster_label, image_data_indexed):
    cluster_selected = cluster_data[cluster_data.cluster_label == cluster_label]
    print(cluster_selected.head())
    
    # Group images into rows of 3
    for i in range(0, len(cluster_selected.image_index), 3):
       # Get up to 3 images for this row
       row_indices = cluster_selected.image_index[i:i+3]
       
       # Create subplot with 1 row and number of images in this row
       fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
       
       # Handle case where there's only one image (axes won't be a list)
       if len(row_indices) == 1:
           axes = [axes]
       
       for j, image_index in enumerate(row_indices):
           #image = samples_indexed[image_index]
           image = image_data_indexed[image_index]
           print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
           
           axes[j].imshow(image, cmap='gray')
           axes[j].axis('off')  # Remove axis labels
           axes[j].set_title(f'Index: {image_index}')
       
       plt.tight_layout()
       plt.show()

In [ ]:
plot_images_from_cluster(cluster_data, 0, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 1, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 2, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 3, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 4, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 5, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 6, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 7, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 8, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 9, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 10, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 11, reconstructions_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 0, samples_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 1, samples_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 2, samples_indexed)

In [ ]:
plot_images_from_cluster(cluster_data, 3, samples_indexed)

In [ ]:
cluster_1 = cluster_data[cluster_data.cluster_label == 1]
cluster_1.head()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 1]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 2]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 3]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 4]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 5]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 6]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 7]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 8]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 9]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 10]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 11]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 12]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 13]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 14]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 15]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 16]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 17]
print(cluster_selected.head())

# Group images into rows of 3
for i in range(0, len(cluster_selected.image_index), 3):
   # Get up to 3 images for this row
   row_indices = cluster_selected.image_index[i:i+3]
   
   # Create subplot with 1 row and number of images in this row
   fig, axes = plt.subplots(1, len(row_indices), figsize=(15, 5))
   
   # Handle case where there's only one image (axes won't be a list)
   if len(row_indices) == 1:
       axes = [axes]
   
   for j, image_index in enumerate(row_indices):
       image = samples_indexed[image_index]
       print(f"Row {i//3 + 1}, Image {j+1}: {image.shape}")
       
       axes[j].imshow(image, cmap='gray')
       axes[j].axis('off')  # Remove axis labels
       axes[j].set_title(f'Index: {image_index}')
   
   plt.tight_layout()
   plt.show()

In [ ]:
cluster_selected = cluster_data[cluster_data.cluster_label == 4]
print(cluster_selected.head())

label_sum = sum(cluster_selected.class_label)
label_proportion = label_sum/cluster_selected.shape[0]
label_proportion

In [ ]:
def calculate_label_proportion(cluster_selected, label_name):
    label_sum = sum(cluster_selected[label_name])
    label_proportion = label_sum/cluster_selected.shape[0]
    return label_proportion
    

In [ ]:
label_proportions_in_clusters = {}

for cluster_label in cluster_data.cluster_label:
    cluster_selected = cluster_data[cluster_data.cluster_label == cluster_label]
    label_proportion = calculate_label_proportion(cluster_selected, 'class_label')
    label_proportions_in_clusters[cluster_label] = label_proportion
    

In [ ]:
label_proportions_in_clusters

In [ ]:
import matplotlib.pyplot as plt

def plot_bar_chart(data_dict, title="Bar Chart", xlabel="Keys", ylabel="Values", figsize=(12, 6)):
   """
   Create a bar plot from a dictionary with sorted keys.
   
   Args:
       data_dict: Dictionary with keys as x-axis labels and values as bar heights
       title: Plot title
       xlabel: X-axis label
       ylabel: Y-axis label
       figsize: Figure size tuple
   """
   # Sort the dictionary by keys
   sorted_items = sorted(data_dict.items())
   keys = [item[0] for item in sorted_items]
   values = [item[1] for item in sorted_items]
   
   # Create the bar plot
   plt.figure(figsize=figsize)
   plt.bar(keys, values)
   plt.title(title)
   plt.xlabel(xlabel)
   plt.ylabel(ylabel)
   plt.grid(axis='y', alpha=0.3)
   
   # Ensure all keys are shown as tick labels
   plt.xticks(keys)
   
   # Rotate x-axis labels if there are many keys
   if len(keys) > 10:
       plt.xticks(rotation=45)
   
   plt.tight_layout()
   plt.show()

## Usage with your data:
#data = {11: 0.5, 0: 0.2692307692307692, 9: 0.6153846153846154, 2: 0.6666666666666666, 
#       5: 0.43478260869565216, 14: 0.36666666666666664, 4: 0.6, 8: 0.7083333333333334, 
#       3: 0.46153846153846156, 15: 0.18181818181818182, 1: 0.5833333333333334, 
#       13: 0.6666666666666666, 16: 0.5, 7: 0.3333333333333333, 12: 0.8, 
#       10: 0.5833333333333334, 6: 0.3333333333333333}
#
#plot_bar_chart(data, title="Data Distribution", xlabel="Categories", ylabel="Values")

In [ ]:
import matplotlib.pyplot as plt

def plot_bar_chart(data_dict, title="Bar Chart", xlabel="Keys", ylabel="Values", figsize=(12, 6), 
                   title_fontsize=16, label_fontsize=14, tick_fontsize=12):
    """
    Create a bar plot from a dictionary with sorted keys.
    
    Args:
        data_dict: Dictionary with keys as x-axis labels and values as bar heights
        title: Plot title
        xlabel: X-axis label
        ylabel: Y-axis label
        figsize: Figure size tuple
        title_fontsize: Font size for the title
        label_fontsize: Font size for axis labels
        tick_fontsize: Font size for tick labels
    """
    # Sort the dictionary by keys
    sorted_items = sorted(data_dict.items())
    keys = [item[0] for item in sorted_items]
    values = [item[1] for item in sorted_items]
    
    # Create the bar plot
    plt.figure(figsize=figsize)
    plt.bar(range(len(keys)), values)  # Use range for x-positions
    plt.title(title, fontsize=title_fontsize)
    plt.xlabel(xlabel, fontsize=label_fontsize)
    plt.ylabel(ylabel, fontsize=label_fontsize)
    plt.grid(axis='y', alpha=0.3)
    
    # Set x-axis ticks and labels properly for categorical data
    plt.xticks(range(len(keys)), keys, fontsize=tick_fontsize)
    plt.yticks(fontsize=tick_fontsize)
    
    # Rotate x-axis labels if there are many keys
    if len(keys) > 10:
        plt.xticks(range(len(keys)), keys, rotation=45, fontsize=tick_fontsize)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_bar_chart(label_proportions_in_clusters, xlabel='clusters', ylabel='proportion of class label', label_fontsize=18, tick_fontsize=18, figsize=(17, 10))


In [ ]:
def plot_samples(sample_history, samples, epoch_stride=5, fig_scale=3, max_images_per_row=None, title_fontsize=16):
    """
    Plots input, noisy samples (for DAE) and reconstruction.
    Each `epoch_stride`-th epoch
    
    Args:
        max_images_per_row: Maximum number of images to display per row (None = all)
        fig_scale: Scale factor for figure size (increased default from 1 to 3)
        title_fontsize: Font size for the plot title
    """
    single_el_idx = samples['single_el_idx']
    
    # Limit the number of images if specified
    if max_images_per_row is not None:
        single_el_idx = single_el_idx[:max_images_per_row]
    
    images_noisy = samples['images_noisy'][single_el_idx, 0]
    images = samples['images'][single_el_idx, 0]
    
    last_epoch = np.max(list(sample_history.keys()))
    
    for epoch_idx, hist_el in sample_history.items():
        if epoch_idx % epoch_stride != 0 and epoch_idx != last_epoch:
            continue
            
        samples_arr = [images_noisy, hist_el['y'][single_el_idx, 0], images]
        
        ny = len(samples_arr)
        nx = len(samples_arr[0])
        
        # Increased figure size calculation for bigger images
        plt.figure(figsize=(fig_scale*nx*1.5, fig_scale*ny*1.5))
        
        m = mosaic(samples_arr)
        plt.title(f'after epoch {int(epoch_idx)}', fontsize=title_fontsize)  # Customizable title font
        plt.imshow(m, cmap='gray', vmin=-.5, vmax=.5)
        
        plt.tight_layout(pad=0.1, h_pad=0, w_pad=0)
        plt.show()
        plt.close()

In [ ]:
N_SAMPLE = 32
N_VIS_SAMPLE = 1

In [ ]:
from torchvision import datasets, transforms


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

target_transform = transforms.Compose([
    lambda x: torch.LongTensor([x])
])

In [ ]:

# Same interface as MNIST
fashion_train = datasets.FashionMNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=transform, 
    target_transform=target_transform
)

fashion_test = datasets.FashionMNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=transform, 
    target_transform=target_transform
)

In [ ]:


# Step 4: Create DataLoader with your original collate function
train_loader = torch.utils.data.DataLoader(
    fashion_train, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_ae_dataset,  # Your original function using globals
    drop_last=True
)


In [ ]:
fashion_train.data.shape

In [ ]:
valid_loader = torch.utils.data.DataLoader(
    fashion_test, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_ae_dataset,  # Your original function using globals
    drop_last=True
)

In [ ]:
# Diagnostic code for train_loader object
print("=== TRAIN_LOADER Properties ===")
print(f"Type: {type(train_loader)}")
print(f"Length (number of batches): {len(train_loader)}")

# Check all attributes
print(f"\nAttributes: {[attr for attr in dir(train_loader) if not attr.startswith('_')]}")

# Check key properties
print(f"batch_size: {train_loader.batch_size}")
#print(f"shuffle: {train_loader.shuffle}")
print(f"drop_last: {train_loader.drop_last}")
print(f"collate_fn: {train_loader.collate_fn}")
print(f"dataset type: {type(train_loader.dataset)}")
print(f"dataset length: {len(train_loader.dataset)}")

# Test getting one batch
print(f"\n=== BATCH SAMPLE ===")
for batch in train_loader:
   print(f"Batch type: {type(batch)}")
   print(f"Batch length: {len(batch)}")
   
   # Unpack the batch (based on your collate function returning 3 items)
   noisy_img, clean_img, labels = batch
   
   print(f"\nnoisy_img type: {type(noisy_img)}")
   print(f"noisy_img shape: {noisy_img.shape}")
   print(f"noisy_img dtype: {noisy_img.dtype}")
   print(f"noisy_img device: {noisy_img.device}")
   
   print(f"\nclean_img type: {type(clean_img)}")
   print(f"clean_img shape: {clean_img.shape}")
   print(f"clean_img dtype: {clean_img.dtype}")
   print(f"clean_img device: {clean_img.device}")
   
   print(f"\nlabels type: {type(labels)}")
   print(f"labels shape: {labels.shape}")
   print(f"labels dtype: {labels.dtype}")
   print(f"labels device: {labels.device}")
   
   break  # Only test first batch

In [ ]:
for batch_idx, (noisy_data, data, target) in enumerate(train_loader):
    print('batch_idx:')
    print(batch_idx)
    print('type of noisy_data:')
    print(type(noisy_data))
    print('type of data:')
    print(type(data))
    print('dimensions of data:')
    print(data.data.shape)
    print('type of target:')
    print(type(target))
    print('dimensions of target:')
    print(target.shape)
    break

In [ ]:
N_EPOCHS = 20

In [ ]:
in_size = (28, 28)

In [ ]:
final_spatial_size = 320 // (2**6)

#in_size = target_size
#in_size

In [ ]:
class ConvolutionalAutoEncoder(AutoEncoder):
    def __init__(self, input_size, code_size):
        self.input_size = list(input_size)
        
        # For 28x28 input with 2 stride=2 layers: 28÷4 = 7x7 final spatial size
        # With 32 channels: final feature map is 32×7×7 = 1568 values
        self.hidden_size = 32 * 7 * 7  # = 1568
        
        self.code_size = code_size
        
        super(ConvolutionalAutoEncoder, self).__init__(input_size, code_size)
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, stride=1), nn.LeakyReLU(negative_slope=0.3),   # 28×28×8
            nn.Conv2d(8, 16, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 14×14×16
            nn.Conv2d(16, 32, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3), # 7×7×32
            
            nn.Flatten(),  # 7×7×32 = 1568 values
            
            nn.Linear(1568, 128), nn.LeakyReLU(negative_slope=0.3),  # 1568 → 128
            nn.Linear(128, code_size),  # 128 → code_size
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(code_size, 1568), nn.LeakyReLU(negative_slope=0.3),  # code_size → 1568
            
            nn.Unflatten(1, (32, 7, 7)),  # 1568 → 32×7×7
            
            nn.ConvTranspose2d(32, 16, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 7→14
            nn.ConvTranspose2d(16, 8, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),   # 14→28
            
            nn.Conv2d(8, 1, 3, padding=1, stride=1), nn.Tanh(),  # Final output: 28×28×1
        )

    def decode(self, z):
        reconstruction = self.decoder(z)
        return reconstruction

In [ ]:
in_size = target_size
model = ConvolutionalAutoEncoder(input_size=in_size, code_size=CODE_SIZE).to(device)

#samples = get_samples(valid_loader)
samples = get_samples(train_loader)

In [ ]:
xns = torch.tensor(samples['images_noisy']).to(device)
print(xns.shape)
zs = model.encode(xns)
ys = model(xns)
print(zs.shape)
print(ys.shape)

In [ ]:
# train the autoencoder model, for N_EPOCHS epochs,
# save history of loss values for training and validation sets,
# history of validation samples evolution, and model weights history,


# This code implements the complete training loop for the autoencoder. 
# Let me break it down:


LR = 0.0009
# Sets the number of training epochs to 50
# Sets the learning rate for the Adam optimizer to 0.0009

model_root = pl.Path(MODEL_NAME)
model_root.mkdir(exist_ok=True)
# Creates a directory path for saving model checkpoints using the MODEL_NAME ('ae_model')
# Makes sure the directory exists (creates it if it doesn't)

optimizer = optim.Adam(model.parameters(), lr=LR)
# Creates an Adam optimizer to update the model parameters
# Adam is an adaptive learning rate optimization algorithm well-suited for deep learning

# implement loss explicitly
loss = nn.MSELoss()
# Defines the loss function as Mean Squared Error (MSE)
# This measures the average squared difference between the reconstructed and target images

# train the model
history = {'loss': [], 'val_loss': [], 'epoch': []}
sample_history = {}
# Creates dictionaries to store training metrics and sample reconstructions
# history tracks training and validation losses across epochs
# sample_history will store sample reconstruction results at each epoch

pbar = tqdm.tqdm(range(N_EPOCHS), postfix=f'epoch 0/{N_EPOCHS}')
# Creates a progress bar for tracking the training process
# Will show the current epoch and update metrics during training

for epoch_idx in pbar:
# Starts the main training loop that runs for N_EPOCHS iterations
    epoch_loss = 0
    model.train()
    # Initializes the epoch loss accumulator
    # Sets the model to training mode (enables dropout, batch normalization updates, etc.)
    
    for batch_idx, (noisy_data, data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(noisy_data)
        loss_value = loss(output, data)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.detach().cpu().item()
    # Iterates through all batches in the training dataset
    # For each batch:
    # 
        # Zeros out previous gradients
        # Passes the noisy input through the model
        # Calculates the MSE loss between the reconstruction and clean data
        # Computes gradients via backpropagation
        # Updates model parameters using the optimizer
        # Accumulates the loss value for epoch-level reporting
    
    epoch_loss /= len(train_loader)
    history['loss'].append(epoch_loss)
    history['epoch'].append(epoch_idx)
    # update progress bar

    # Calculates the average loss for the epoch
    # Records the loss and epoch number in the history

    # evaluate on validation set
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch_idx, (noisy_data, data, target) in enumerate(valid_loader):
            output = model(noisy_data)
            loss_value = loss(output, data)
            val_loss += loss_value.detach().cpu().item()
        val_loss /= len(valid_loader)
        history['val_loss'].append(val_loss)

    # Sets the model to evaluation mode (disables dropout, etc.)
    # Disables gradient calculation for efficiency
    # Computes the validation loss on the entire validation set
    # Records the average validation loss in the history
    
    pbar.set_postfix({'epoch': f'{epoch_idx+1}/{N_EPOCHS}', 'loss':f'{epoch_loss:.4f}', 'val_loss':f'{val_loss:.4f}'})
    # evaluate on samples
    # Updates the progress bar with current epoch, training loss, and validation loss
    
    sample_res = eval_on_samples(model, epoch_idx, samples=samples)
    # This saves the reconstructions and the latent space thanks to
    # the eval_on_samples function where in the application of the 
    # model to the evaluation data the return_z parameter is set 
    # to true: 
    # with torch.no_grad():
    #     yz = ae_model(xns, return_z=True)
    #     yz = [el.detach().cpu().numpy() for el in yz]
# 
    #     y = yz[0]
    #     z = yz[1:]
    
    # The output of eval_on_samples looks like this: 

    # sample_res = {'z': z, 'y': y, 'epoch': epoch}

    
    sample_history[epoch_idx] = sample_res
    # Evaluates the model on the sample images
    # Stores reconstructions for later visualization

    # save model weights
    torch.save({
                'epoch': epoch_idx,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss
                }, model_root/f'model_{epoch_idx:03d}.pth')

    # Saves a checkpoint of the model at each epoch
    # The checkpoint includes:
    # 
    # Current epoch number
    # Model parameters
    # Optimizer state (allows resuming training)
    # Loss function
    # 
    # 
    # Uses a formatted filename with padded epoch number (e.g., 'model_001.pth')
# 
# This is a complete training pipeline that not only trains the model 
# but also tracks metrics, evaluates on validation data, 
# and creates visualizations to monitor progress - 
# all while saving checkpoints for later analysis or resuming training.
    

In [ ]:
plot_hist(history)

In [ ]:
plot_samples(sample_history, samples=samples, epoch_stride=5, fig_scale=7, max_images_per_row=4,  title_fontsize=60)


In [ ]:
from torch.utils.data import Subset
from collections import defaultdict
import random

# Group indices by class
class_indices = defaultdict(list)
for i, (_, label) in enumerate(fashion_train):
    class_indices[label.item()].append(i)

# Take half from each class
subset_indices = []
for class_label, indices in class_indices.items():
    tenth_class_size = len(indices) // 30
    random.shuffle(indices)
    subset_indices.extend(indices[:tenth_class_size])

# Create subset
subset_fashion_train = Subset(fashion_train, subset_indices)

In [ ]:
# For subset, you need to access through the underlying dataset
print(f"Subset underlying data shape: {subset_fashion_train.dataset.data.shape}")
print(f"Subset underlying targets shape: {subset_fashion_train.dataset.targets.shape}")

# To get the actual subset data/targets:
subset_data = subset_fashion_train.dataset.data[subset_fashion_train.indices]
subset_targets = subset_fashion_train.dataset.targets[subset_fashion_train.indices]
print(f"Actual subset data shape: {subset_data.shape}")
print(f"Actual subset targets shape: {subset_targets.shape}")

In [ ]:
# Step 4: Create DataLoader with your original collate function
train_loader = torch.utils.data.DataLoader(
    subset_fashion_train, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_ae_dataset,  # Your original function using globals
    drop_last=True
)


In [ ]:
class ConvolutionalAutoEncoder(AutoEncoder):
    def __init__(self, input_size, code_size):
        self.input_size = list(input_size)
        
        # For 28x28 input with 2 stride=2 layers: 28÷4 = 7x7 final spatial size
        # With 32 channels: final feature map is 32×7×7 = 1568 values
        self.hidden_size = 32 * 7 * 7  # = 1568
        
        self.code_size = code_size
        
        super(ConvolutionalAutoEncoder, self).__init__(input_size, code_size)
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, stride=1), nn.LeakyReLU(negative_slope=0.3),   # 28×28×8
            nn.Conv2d(8, 16, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 14×14×16
            nn.Conv2d(16, 32, 3, padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3), # 7×7×32
            
            nn.Flatten(),  # 7×7×32 = 1568 values
            
            nn.Linear(1568, 128), nn.LeakyReLU(negative_slope=0.3),  # 1568 → 128
            nn.Linear(128, code_size),  # 128 → code_size
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(code_size, 1568), nn.LeakyReLU(negative_slope=0.3),  # code_size → 1568
            
            nn.Unflatten(1, (32, 7, 7)),  # 1568 → 32×7×7
            
            nn.ConvTranspose2d(32, 16, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),  # 7→14
            nn.ConvTranspose2d(16, 8, 3, padding=1, output_padding=1, stride=2), nn.LeakyReLU(negative_slope=0.3),   # 14→28
            
            nn.Conv2d(8, 1, 3, padding=1, stride=1), nn.Tanh(),  # Final output: 28×28×1
        )

    def decode(self, z):
        reconstruction = self.decoder(z)
        return reconstruction

In [ ]:
in_size = target_size
model = ConvolutionalAutoEncoder(input_size=in_size, code_size=CODE_SIZE).to(device)

#samples = get_samples(valid_loader)
samples = get_samples(train_loader)

In [ ]:
xns = torch.tensor(samples['images_noisy']).to(device)
print(xns.shape)
zs = model.encode(xns)
ys = model(xns)
print(zs.shape)
print(ys.shape)

In [ ]:
# train the autoencoder model, for N_EPOCHS epochs,
# save history of loss values for training and validation sets,
# history of validation samples evolution, and model weights history,


# This code implements the complete training loop for the autoencoder. 
# Let me break it down:


LR = 0.0009
# Sets the number of training epochs to 50
# Sets the learning rate for the Adam optimizer to 0.0009

model_root = pl.Path(MODEL_NAME)
model_root.mkdir(exist_ok=True)
# Creates a directory path for saving model checkpoints using the MODEL_NAME ('ae_model')
# Makes sure the directory exists (creates it if it doesn't)

optimizer = optim.Adam(model.parameters(), lr=LR)
# Creates an Adam optimizer to update the model parameters
# Adam is an adaptive learning rate optimization algorithm well-suited for deep learning

# implement loss explicitly
loss = nn.MSELoss()
# Defines the loss function as Mean Squared Error (MSE)
# This measures the average squared difference between the reconstructed and target images

# train the model
history = {'loss': [], 'val_loss': [], 'epoch': []}
sample_history = {}
# Creates dictionaries to store training metrics and sample reconstructions
# history tracks training and validation losses across epochs
# sample_history will store sample reconstruction results at each epoch

pbar = tqdm.tqdm(range(N_EPOCHS), postfix=f'epoch 0/{N_EPOCHS}')
# Creates a progress bar for tracking the training process
# Will show the current epoch and update metrics during training

for epoch_idx in pbar:
# Starts the main training loop that runs for N_EPOCHS iterations
    epoch_loss = 0
    model.train()
    # Initializes the epoch loss accumulator
    # Sets the model to training mode (enables dropout, batch normalization updates, etc.)
    
    for batch_idx, (noisy_data, data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(noisy_data)
        loss_value = loss(output, data)
        loss_value.backward()
        optimizer.step()
        epoch_loss += loss_value.detach().cpu().item()
    # Iterates through all batches in the training dataset
    # For each batch:
    # 
        # Zeros out previous gradients
        # Passes the noisy input through the model
        # Calculates the MSE loss between the reconstruction and clean data
        # Computes gradients via backpropagation
        # Updates model parameters using the optimizer
        # Accumulates the loss value for epoch-level reporting
    
    epoch_loss /= len(train_loader)
    history['loss'].append(epoch_loss)
    history['epoch'].append(epoch_idx)
    # update progress bar

    # Calculates the average loss for the epoch
    # Records the loss and epoch number in the history

    # evaluate on validation set
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch_idx, (noisy_data, data, target) in enumerate(valid_loader):
            output = model(noisy_data)
            loss_value = loss(output, data)
            val_loss += loss_value.detach().cpu().item()
        val_loss /= len(valid_loader)
        history['val_loss'].append(val_loss)

    # Sets the model to evaluation mode (disables dropout, etc.)
    # Disables gradient calculation for efficiency
    # Computes the validation loss on the entire validation set
    # Records the average validation loss in the history
    
    pbar.set_postfix({'epoch': f'{epoch_idx+1}/{N_EPOCHS}', 'loss':f'{epoch_loss:.4f}', 'val_loss':f'{val_loss:.4f}'})
    # evaluate on samples
    # Updates the progress bar with current epoch, training loss, and validation loss
    
    sample_res = eval_on_samples(model, epoch_idx, samples=samples)
    # This saves the reconstructions and the latent space thanks to
    # the eval_on_samples function where in the application of the 
    # model to the evaluation data the return_z parameter is set 
    # to true: 
    # with torch.no_grad():
    #     yz = ae_model(xns, return_z=True)
    #     yz = [el.detach().cpu().numpy() for el in yz]
# 
    #     y = yz[0]
    #     z = yz[1:]
    
    # The output of eval_on_samples looks like this: 

    # sample_res = {'z': z, 'y': y, 'epoch': epoch}

    
    sample_history[epoch_idx] = sample_res
    # Evaluates the model on the sample images
    # Stores reconstructions for later visualization

    # save model weights
    torch.save({
                'epoch': epoch_idx,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss
                }, model_root/f'model_{epoch_idx:03d}.pth')

    # Saves a checkpoint of the model at each epoch
    # The checkpoint includes:
    # 
    # Current epoch number
    # Model parameters
    # Optimizer state (allows resuming training)
    # Loss function
    # 
    # 
    # Uses a formatted filename with padded epoch number (e.g., 'model_001.pth')
# 
# This is a complete training pipeline that not only trains the model 
# but also tracks metrics, evaluates on validation data, 
# and creates visualizations to monitor progress - 
# all while saving checkpoints for later analysis or resuming training.
    

In [ ]:
plot_hist(history)

In [ ]:
plot_samples(sample_history, samples=samples, epoch_stride=5, fig_scale=7, max_images_per_row=4, title_fontsize=60)
